In [1]:
import pandas as pd

In [2]:
# location from which scraped data will be retrieved
INPUT_FILE = './ScrapedMatchData/match_info_{}.json'

In [3]:
# retrieve number of files
with open('./ScrapedMatchData/NumFiles.txt','r') as NumFile:
    number_of_files = int(NumFile.read())

In [7]:
# retrieve number of new files
with open('./ScrapedMatchData/NumNewFiles.txt','r') as NumFile:
    newFileNum = int(NumFile.read())

In [5]:
# store all file names
file_names = [INPUT_FILE.format(i) for i in range(1,number_of_files+1)]

In [8]:
# try to find out what each element represents

# Unsure about 0th element
# 1st element is weight class
# 3rd/4th elements are type of victory (fall, decision, etc.)
# 7th element is winner's score, 10th element is loser score
# 16th thru 23rd elements are winner's and loser's names and schools
# 31st element is round of match if tournament (quarterfinals, semifinals, etc.)
# 40th element is numbers 1-16, or nan/none, could be tournament seed?
# appended last entries to be event Name, event Id, dual Id, event date

In [18]:
# Load all match data (takes a couple minutes)
frames = [pd.read_json(file_name) for file_name in file_names]

In [19]:
# Tracking order of events (should be redundant now that collecting date info)
for i,frame in enumerate(frames,1): # preserves order of events (scraped from newest to oldest)
    frame["Matches Passed"] = i

In [20]:
# Combine into one dataframe
all_match_data = pd.concat(frames)

In [35]:
# all_match_data

In [36]:
all_match_data.to_csv('all_match_data.csv')

In [37]:
# rows do not all line up, kinda sucks
# Look into splitting/shifting data (maybe using event names files since all blank names are duals?)

In [38]:
# Original size of raw data
all_match_data.shape

(20819, 47)

In [39]:
# Drop exactly duplicated rows (from webscraping process)
# Note: Might be more duplicates that just aren't exact (from website setup), need to check further
all_match_data.drop_duplicates(inplace=True)

# Size after dropping duplicated rows
all_match_data.shape

(20819, 47)

In [40]:
# Need to fix below so we stop throwing away data and have representation for duals

In [45]:
# Define set of 15 official New York State Public High School weight classes

weight_classes = {99, 106, 113, 120, 126, 132, 138, 145, 152, 160, 170, 182, 195, 220, 285}

In [46]:
# Define set of improper (i.e. non-integer) weight classes, the other unique values of col 1
# alt_weight_classes = set(all_match_data[1].unique()[15:])

In [47]:
# For now, filter table to only include rows with proper weight classes
weight_filter = [x in weight_classes for x in all_match_data[1]]
weight_filtered_data = all_match_data.loc[weight_filter,:]

In [48]:
# Data size after selecting only 'proper' weight classes
weight_filtered_data.shape

(18362, 47)

In [49]:
# Rename columns we know by inspection
weight_filtered_data.rename(columns={1:"Weight Class", 3:"Victory Type (L)", 4:"Victory Type (S)",
                         7:"Winner Score", 10:"Loser Score", 16:"Winner First Name",
                         17:"Winner Last Name", 18:"Winner School (L)", 19:"Winner School (S)",
                         20:"Loser First Name", 21:"Loser Last Name", 22:"Loser School (L)",
                         23:"Loser School (S)", 29: "Unknown (Seed?)", 31:"Round", 42:"Event Name", 43:"Event ID",
                         45:"Event Date"},
                      inplace=True
                    )

C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\pandas\core\frame.py:4130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [51]:
# Ensure weight classes and event date are cast as integers
weight_filtered_data['Weight Class'] = [int(x) for x in weight_filtered_data['Weight Class']]
weight_filtered_data['Event Date'] = [int(x) for x in weight_filtered_data['Event Date']]

C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
# Dropping empty, redundant, and/or unintelligible columns (decided by inspection)
drop_cols = [0,2,5,6,8,9,11,12,13,14,15,24,25,26,27,28,30,32,33,34,35,36,37,38,39,40,41,44]
weight_filtered_data.drop(columns=drop_cols,inplace=True)

C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\pandas\core\frame.py:3994: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
# Reset index so matches now have unique IDs
weight_filtered_data.reset_index(drop=True,inplace=True)

In [55]:
# Fix Event ID to be string instead of float
weight_filtered_data['Event ID'] = weight_filtered_data['Event ID'].astype(str).replace('\.0', '', regex=True)

C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
### Clean name columns 
name_cols = ['Winner First Name','Winner Last Name','Loser First Name','Loser Last Name']

for name in name_cols:
    
    col = weight_filtered_data[name]
    col = [x.strip().title() if type(x)==str else x for x in col] # Removes extra whitespace, ensure proper capitalization
    weight_filtered_data[name] = col

C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
# Create full names for winners and losers from their cleaned first and last names
weight_filtered_data['Winner Full Name'] = weight_filtered_data['Winner First Name'] + ' ' + weight_filtered_data['Winner Last Name']
weight_filtered_data['Loser Full Name'] = weight_filtered_data['Loser First Name'] + ' ' + weight_filtered_data['Loser Last Name']

C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\colum\Anaconda3\envs\webscraping\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
# Save weight_filtered_data to csv file named MATCHES.csv
weight_filtered_data.to_csv('MATCHES.csv') # Remember: currently only using rows with 'proper' weight classes

In [59]:
# union of winner/loser full names is set of all wrestlers in dataset
wrestlers = set(weight_filtered_data['Winner Full Name']) | set(weight_filtered_data['Loser Full Name'])
wrestlers = [x for x in wrestlers if x==x] # remove nan, convert to list

In [65]:
# create infoscrape function that goes through dataset and collects desired info for each wrestler

def infoscrape(fullname,df):
    '''infoscrape function receives full name of wrestler and matches dataframe
    and collects wrestler info from dataset'''

    # Initialize values of interest
    weight_class = 0
    wins = 0
    losses = 0
    matches = 0
    school = ''
    school_code = ''
    first_name = ''
    last_name = ''
    
    # Find observations corresponding to wrestler name
    win_id = df['Winner Full Name'] == fullname
    loss_id = df['Loser Full Name'] == fullname
    winning_matches = df.loc[win_id,:]
    losing_matches = df.loc[loss_id,:]
    
    # Split full name
    first_name, last_name = fullname.split(' ',1)
    
    # Counting stats (should check if names show in correct columns for forfeits, byes, etc.)
    wins = sum(win_id)
    losses = sum(loss_id)
    matches = wins+losses
    
    # Extract weight class, school, etc.
    win_weight = winning_matches['Weight Class'].unique()
    loss_weight = losing_matches['Weight Class'].unique()
    
    if win_weight.size > 0: # Avoiding 'if win_weight:' because it gives truth amibiguity warning
        weight_class = int(win_weight[0])
    else: # !!!Still need to add consideration for multiple weight classes!!!
        weight_class = int(loss_weight[0])
        
    win_school = winning_matches['Winner School (L)'].unique()
    win_school_code = winning_matches['Winner School (S)'].unique()
    loss_school = losing_matches['Loser School (L)'].unique()
    loss_school_code = losing_matches['Loser School (S)'].unique()
    
    if win_school.size > 0: # Avoiding 'if win_school:' because it gives truth amibiguity warning
        school = win_school[0]
        school_code = win_school_code[0]
    else: 
        school = loss_school[0]
        school_code = loss_school_code[0]
        
    # Return list of extracted data 
    return({'First Name':first_name,'Last Name':last_name,'Full Name':fullname,
            'School Name':school,'School Code':school_code,
            'Weight Class':weight_class,'Wins':wins,'Losses':losses,'Matches':matches})

In [66]:
# Construct wrestler dataframe (takes a minute)
wrestler_data = [infoscrape(wrestler,weight_filtered_data) for wrestler in wrestlers]
WRESTLERS = pd.DataFrame(wrestler_data)

In [67]:
# Save WRESTLERS to csv file named WRESTLERS.csv
WRESTLERS.to_csv('WRESTLERS.csv')